# Calculating Beam-Size Through Telescope

In many situations in the lab we find ourselves needing a Gaussian beam of a given set of qualities, and must compute how we can get them by selecting a telescope to work with. This program will serve as an example implementation of the functions in the `beam-doctor` [GitHub repository](https://github.com/lwcooke/beam-doctor). 

The original purpose of this code was to compute the ideal set of lenses, and the distance between them, for correcting the beam output of lasers. The output beams are typically elliptical (especially if amplified by a tapered-amplifier), so it computes lenses for the $x$ and $y$ dimensions seperately, assuming one can use cylindrical lenses to independently correct both.

We will define some constraints on the desired output beam size & divergence, and then based on the input beam parameters we will compute the telescope configurations that will produce such a beam (from a selection of lenses). The initial beam parameters are measured by a beam profiler, which is common lab device for measuring spatial profiles of beams; we require the beam diameter at several distances, to fit for the divergence angle. 

In [1]:
#----- Import Libraries -----
import numpy as np #Of course
from prettytable import PrettyTable #Fancy-looking output
from scipy.stats import linregress #Fitting the initial beam
import BeamDoctorFunctions as bdf #beam-doctor function library

## Compute Starting Beam Size/Divergence

Using a list of data points measured with the beam profiler, we will use a linear fit to determine the starting beam divergence. The divergence half-angle can be found from the slope of the beam radii as,

$$ \theta = \arctan m, $$

Where $m$ is the slope of the line. We can use the `beamprofiler` function for this.

In [2]:
#----- Beam Data -----
d = np.array([ 0, 50, 100, 240 ]) #Measured distances [mm]
rx = (1/2)*np.array([ 0.806, 0.810, 0.805, 0.820 ]) #x-radius from diameters [mm]
ry = (1/2)*np.array([ 0.735, 0.730, 0.705, 0.740 ]) #y-radius from diameters [mm]

#----- Fit Data -----
thetax, thetay = bdf.beamprofiler( rx, d ), bdf.beamprofiler( ry, d )

print(f'theta_x: {thetax*180/np.pi:.2g} deg, theta_y: {thetay*180/np.pi:.2g} deg' )

theta_x: 0.0016 deg, theta_y: 0.00078 deg


## Try Telescope Configurations

Now we will use the beam profile from above to determine the optimal telescope for us, based on the lenses available. There are several functions with fairly complete documentation, but we will describe the general function of them here.

1. `telescope`: For a given set of focal lengths, separation distance, and input beam profile ($1$D) this function computes the output beam profile.
2. `lensconfigs`: Computes a list of all possible lens configurations to try, from the input choices of focal lengths. It has a parameter to also try separation distances on either side of the typical $d = f_1 + f_2$, in case the input beam is not collimated.
3. `tryall`: From the initial beam profile this function will call the `telescope` function to compute the output profiles for all lens configuration in an input list (typically from the `lensconfigs` function).
4. `filterall`: Sorts through a list of given beam profiles and lens configurations to remove any which exceed an input set of tolerances. This function thus allows us to specify the maximum beam size and divergence we want, and removes all of the attempted configurations which exceed these specifications.
5. `beamdoctor`: From the input beam sizes, this function just calls all the previous functions as they were written to return a well-formatted table of final choices of telescopes that meet specifications.

### Compute Telescopes
With the above functions we can now establish a lens inventory and set our output beam specifications, and then see what telescopes meet those requirements. 

Generally speaking, if we aren't doing anything crazy near the diffraction limit or with insane sensitivity, we prefer to use negative focal length lenses; this reduces the separation distance, which is good because real-estate on optics benches isn't cheap (location, location, location!). Given this, the order of the lenses matters depending on whether we are looking to do beam expansion or reduction (even if it's slight). We sumarize the choices below:
1. **Beam Expansion:** The negative focal length lens (concave) should come first. The negative lens will cause the beam to diverge onto the positive lens, which will then straighten it out at the larger size.
2. **Beam Reduction:** The negative focal length lens (convex) comes first. The positive focal length will focus the beam down to a small diameter onto the negative lens, which will then straighten it out at the smaller size.

In [9]:
#----- Lens Choices -----
#Thorlabs cylindrical lens inventories
cylvex = [ 50, 75, 100, 150, 200, 250, 300, 400, 500, 1000] #Convex [mm]
cylcave = [ -50, -75, -100, -150, -200, -400, -1000] #Concave [mm]

In [10]:
#----- Initial Beam Profiles -----
rx_in, ry_in = np.array([ rx[0], thetax ]), np.array([ ry[0], thetay ]) #Input [radius, half-angle]

#--- Set Tolerances ---
rmin, rmax = 0.6, 1.2 #Min/Max tolerable beam radius [mm]
thetamax = 0.002*(np.pi/180) #Maximum tolerable beam divergence [rad]

In [11]:
#----- Configurations for x-profile -----
print('---------- X BEAM PROFILES ----------')
bdf.beamdoctor(rx_in, cylcave, cylvex, rmax, thetamax, dpm=100, rmin=rmin)

---------- X BEAM PROFILES ----------
+-------------+-------------+---------+---------+--------+
| Radius [mm] | Theta [deg] | f1 [mm] | f2 [mm] | d [mm] |
+-------------+-------------+---------+---------+--------+
|    0.605    |    0.001    |   -50   |    75   |   25   |
|    0.807    |    0.001    |   -50   |   100   |   50   |
|    0.808    |    0.001    |   -75   |   150   |   75   |
|    1.078    |    0.001    |   -75   |   200   |  125   |
|    0.606    |    0.001    |   -100  |   150   |   50   |
|    0.809    |    0.001    |   -100  |   200   |  100   |
|    1.012    |    0.001    |   -100  |   250   |  150   |
|    0.674    |    0.001    |   -150  |   250   |  100   |
|    0.810    |    0.001    |   -150  |   300   |  150   |
|    1.082    |    0.001    |   -150  |   400   |  250   |
|    0.607    |    0.001    |   -200  |   300   |  100   |
|    0.812    |    0.001    |   -200  |   400   |  200   |
|    1.016    |    0.001    |   -200  |   500   |  300   |
|    0.983    |   

In [13]:
#----- Configurations for y-profile -----
print('---------- Y BEAM PROFILES ----------')
bdf.beamdoctor(ry_in, cylcave, cylvex, rmax, thetamax, dpm=100, rmin=rmin)

---------- Y BEAM PROFILES ----------
+-------------+-------------+---------+---------+--------+
| Radius [mm] | Theta [deg] | f1 [mm] | f2 [mm] | d [mm] |
+-------------+-------------+---------+---------+--------+
|    0.736    |    0.000    |   -50   |   100   |   50   |
|    1.104    |    0.000    |   -50   |   150   |  100   |
|    0.736    |    0.000    |   -75   |   150   |   75   |
|    0.982    |    0.000    |   -75   |   200   |  125   |
|    0.736    |    0.000    |   -100  |   200   |  100   |
|    0.921    |    0.000    |   -100  |   250   |  150   |
|    1.105    |    0.000    |   -100  |   300   |  200   |
|    0.614    |    0.000    |   -150  |   250   |  100   |
|    0.737    |    0.000    |   -150  |   300   |  150   |
|    0.983    |    0.000    |   -150  |   400   |  250   |
|    0.738    |    0.000    |   -200  |   400   |  200   |
|    0.923    |    0.000    |   -200  |   500   |  300   |
|    0.890    |    -0.002   |   -400  |   1000  |  560   |
|    0.908    |   